In [1]:
import riskfolio as rf

import matplotlib.pyplot as plt
import numpy as np
import pickle
import random

import pandas as pd

import gym
from gym import spaces

from stable_baselines3 import PPO, A2C, TD3, SAC, DDPG, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecFrameStack, SubprocVecEnv
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

import torch

In [2]:
def softmax_normalization(actions):
    numerator = np.exp(actions)
    denominator = np.sum(np.exp(actions))
    softmax_output = numerator/denominator
    return softmax_output

## Definition of gym Environment

In [3]:
# with open('data/sp500_it_components_7y.pickle', 'rb') as f:
#     test_data = pickle.load(f)
# test_data[0]['corrs']

In [66]:
class PortfolioAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, data_file,
                 objective = 'max_return',
                 risk_measure = 'MV',
                 observations_range = (0, 1259)):
        """
        Инициализация среды
        """
        super(PortfolioAllocationEnv, self).__init__()

        with open(data_file, 'rb') as f:
            self.data = pickle.load(f)
#             for d in self.data:
#                 d['corrs'] = d['corrs'].iloc[3:, :]

        self.stock_dim = self.data[0]['corrs'].shape[1]

        close_price_changes = {stock['ticker']: stock['data']['Close_change'] for stock in self.data}
        self.close_prices_pct = pd.DataFrame(close_price_changes)

        self.tickers = [stock['ticker'] for stock in self.data]
        self.n_assets = len(self.data)

        self.objective = objective
        self.risk_measure = risk_measure

        self.rewards_memory = None
        self.actions_memory = None

        self.portfolio_value = 0
        self.observations_range = observations_range
        self.current_step = observations_range[0]

        self.weights = np.ones(self.n_assets) / self.n_assets

        self.reward_range = (-10, 10)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(self.n_assets,))
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets, self.stock_dim + 1))

    def step(self, action):
        """
        Выполнение одного шага
        """
        # assert self.action_space.contains(action), "Invalid action"

        # Получаем текущее состояние
        state = self._get_state()

        # Вычисляем награду
        reward = self._get_reward(action)

        # Обновляем текущий шаг
        self.current_step += 1

        # Проверяем, достигнут ли конец эпизода
        done = self.current_step >= self.observations_range[1]

        if done:
            print(self.render())

        # Возвращаем состояние, награду и флаг окончания эпизода
        return state, reward, done, {}

    def reset(self):
        """
        Сброс среды в начальное состояние
        """
        self.portfolio_value = 0
        self.rewards_memory = []
        self.current_step = self.observations_range[0]
        self.actions_memory = []
        self.weights = np.ones(self.n_assets) / self.n_assets
        return self._get_state()

    def render(self, mode='human'):
        """
        Вывод информации о среде
        """
        return {
            'min_reward': min(self.rewards_memory),
            'max_reward': max(self.rewards_memory),
            'mean_reward': np.array(self.rewards_memory).mean(),
            'portfolio_value': self.portfolio_value,
            'weights_q0.25': np.quantile(self.weights, 0.25),
            'weights_q0.5': np.quantile(self.weights, 0.5),
            'weights_q0.75': np.quantile(self.weights, 0.75),
            'weights_max': np.max(self.weights),
            'weights_std': self.weights.std()
        }

    def _get_state(self):
        """
        Получение текущего состояния
        """
        values = [stock['corrs'].iloc[self.current_step].values for stock in self.data]
        observation = np.vstack(values)
        observation = np.concatenate([observation, self.weights[:, None]], axis=1).astype(np.float32)
        observation = np.nan_to_num(observation)
        return observation

    def _get_reward(self, action):
        """
        Вычисление награды
        """

        self.weights += action
        self.weights = softmax_normalization(self.weights)

        self.actions_memory.append(action)

        prices_change_hist = self.close_prices_pct.iloc[self.observations_range[0]:self.current_step + 1]
        current_prices = self.close_prices_pct.iloc[self.current_step].values

        portfolio_value_change = current_prices @ self.weights
        self.portfolio_value += portfolio_value_change

        reward = 0

        if self.objective == 'sharpe':
            mu = rf.mean_vector(prices_change_hist, method='hist')
            cov = rf.covar_matrix(prices_change_hist, method='shrunk')
            reward = rf.Sharpe(self.weights, mu, cov, prices_change_hist, rm=self.risk_measure)
        elif self.objective == 'price_change':
            # reward = portfolio_value_change
            reward = portfolio_value_change
        elif self.objective == 'min_risk':
            mu = rf.mean_vector(prices_change_hist, method='hist')
            cov = rf.covar_matrix(prices_change_hist, method='shrunk')
            reward = -rf.Sharpe_Risk(self.weights, cov, mu, rm=self.risk_measure)

        if np.isinf(reward):
            reward = 0

        self.rewards_memory.append(reward)
        # print('act', self.current_step, np.isnan(action).sum(), np.array(self.rewards_memory).mean())

        return reward

In [79]:
# class PortfolioAllocationEnv(gym.Env):
#     metadata = {'render.modes': ['human']}

#     def __init__(self, data_file,
#                  sector = None,
#                  objective = 'max_return',
#                  risk_measure = 'MV',
#                  drop_close = True,
#                  observations_range = (0, 1259)):
#         """
#         Инициализация среды
#         """
#         super(PortfolioAllocationEnv, self).__init__()

#         with open(data_file, 'rb') as f:
#             self.data = pickle.load(f)
#             if sector is not None:
#                 self.data = list(filter(lambda stock: stock['sector'] == sector, self.data))
#             if drop_close:
#                 for stock in self.data:
#                     stock['data'] = stock['data'].drop(['Close'], axis=1)

#         self.stock_dim = self.data[0]['data'].shape[1]

#         close_price_changes = {stock['ticker']: stock['data']['Close_Change'] for stock in self.data}
#         self.close_prices_pct = pd.DataFrame(close_price_changes)

#         self.tickers = [stock['ticker'] for stock in self.data]
#         self.n_assets = len(self.data)

#         self.objective = objective
#         self.risk_measure = risk_measure

#         self.rewards_memory = None
#         self.actions_memory = None

#         self.portfolio_value = 0
#         self.observations_range = observations_range
#         self.current_step = observations_range[0]

#         self.weights = np.ones(self.n_assets) / self.n_assets

#         self.reward_range = (-10, 10)
#         self.action_space = gym.spaces.Box(low=-1, high=1, shape=(self.n_assets,))
#         self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets, self.stock_dim + 1))

#     def step(self, action):
#         """
#         Выполнение одного шага
#         """
#         # assert self.action_space.contains(action), "Invalid action"

#         # Получаем текущее состояние
#         state = self._get_state()

#         # Вычисляем награду
#         reward = self._get_reward(action)

#         # Обновляем текущий шаг
#         self.current_step += 1

#         # Проверяем, достигнут ли конец эпизода
#         done = self.current_step >= self.observations_range[1]

#         if done:
#             print(self.render())

#         # Возвращаем состояние, награду и флаг окончания эпизода
#         return state, reward, done, {}

#     def reset(self):
#         """
#         Сброс среды в начальное состояние
#         """
#         self.portfolio_value = 0
#         self.rewards_memory = []
#         self.current_step = self.observations_range[0]
#         self.actions_memory = []
#         self.weights = np.ones(self.n_assets) / self.n_assets
#         return self._get_state()

#     def render(self, mode='human'):
#         """
#         Вывод информации о среде
#         """
#         return {
#             'min_reward': min(self.rewards_memory),
#             'max_reward': max(self.rewards_memory),
#             'mean_reward': np.array(self.rewards_memory).mean(),
#             'portfolio_value': self.portfolio_value,
#             'weights_q0.25': np.quantile(self.weights, 0.25),
#             'weights_q0.5': np.quantile(self.weights, 0.5),
#             'weights_q0.75': np.quantile(self.weights, 0.75),
#             'weights_max': np.max(self.weights),
#             'weights_std': self.weights.std()
#         }

#     def _get_state(self):
#         """
#         Получение текущего состояния
#         """
#         values = [stock['data'].iloc[self.current_step].values for stock in self.data]
#         observation = np.vstack(values)
#         observation = np.concatenate([observation, self.weights[:, None]], axis=1).astype(np.float32)
#         observation = np.nan_to_num(observation)
#         return observation

#     def _get_reward(self, action):
#         """
#         Вычисление награды
#         """

#         self.weights += action
#         self.weights = softmax_normalization(self.weights)

#         self.actions_memory.append(action)

#         prices_change_hist = self.close_prices_pct.iloc[self.observations_range[0]:self.current_step + 1]
#         current_prices = self.close_prices_pct.iloc[self.current_step].values

#         portfolio_value_change = current_prices @ self.weights
#         self.portfolio_value += portfolio_value_change

#         reward = 0

#         if self.objective == 'sharpe':
#             mu = rf.mean_vector(prices_change_hist, method='hist')
#             cov = rf.covar_matrix(prices_change_hist, method='shrunk')
#             reward = rf.Sharpe(self.weights, mu, cov, prices_change_hist, rm=self.risk_measure)
#         elif self.objective == 'price_change':
#             # reward = portfolio_value_change
#             reward = portfolio_value_change
#         elif self.objective == 'min_risk':
#             mu = rf.mean_vector(prices_change_hist, method='hist')
#             cov = rf.covar_matrix(prices_change_hist, method='shrunk')
#             reward = -rf.Sharpe_Risk(self.weights, cov, mu, rm=self.risk_measure)

#         if np.isinf(reward):
#             reward = 0

#         self.rewards_memory.append(reward)
#         # print('act', self.current_step, np.isnan(action).sum(), np.array(self.rewards_memory).mean())

#         return reward

In [80]:
train_env = PortfolioAllocationEnv(data_file='data/sp500_re_components_7y.pickle',
#                                    sector='Information Technology',
                                   objective='price_change',
#                                    drop_close=False,
                                   observations_range=(0, 1260))  # 252 x 5 = 1260, 252 x 3 = 756
check_env(train_env, skip_render_check=False)

/home/hivaze/Documents/CODE-W/PyCharm Projects/portfolio_optimization/venv/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:213: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [81]:
len(train_env.data[0]['corrs'])

1760

### Vectorization of Env

In [82]:
from stable_baselines3.common.vec_env import VecCheckNan

train_env = DummyVecEnv([lambda: train_env])
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=False)
train_env = VecCheckNan(train_env, raise_exception=True)
# train_env = VecFrameStack(train_env, 15, channels_order='first')

In [83]:
train_env.observation_space.shape, train_env.action_space.shape

((28, 41), (28,))

## stablebaselines Models

In [84]:
from stable_baselines3.ppo import MlpPolicy
from sb3_contrib.ppo_recurrent import MlpLstmPolicy
from sb3_contrib import RecurrentPPO

In [53]:
DDPG?

In [85]:
model = SAC('MlpPolicy', train_env, learning_rate=1e-3, buffer_size=20_000,
            verbose=1, tensorboard_log='logs/',
            device='cuda', batch_size=128,
#             policy_kwargs={'net_arch': [800, 400]}
            )
model.policy

Using cuda device


SACPolicy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (latent_pi): Sequential(
      (0): Linear(in_features=1148, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
    (mu): Linear(in_features=256, out_features=28, bias=True)
    (log_std): Linear(in_features=256, out_features=28, bias=True)
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=1176, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
    (qf1): Sequential(
      (0): Linear(in_features=1176, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_feature

In [86]:
mean_reward, std_reward = evaluate_policy(model, train_env, n_eval_episodes=1, deterministic=False, warn=False, return_episode_rewards=True)
print(mean_reward, std_reward)

{'min_reward': -0.17888513111263077, 'max_reward': 0.1016405990048536, 'mean_reward': 0.00027419362430891305, 'portfolio_value': 0.3454839666292308, 'weights_q0.25': 0.019981438691910217, 'weights_q0.5': 0.025651049657377967, 'weights_q0.75': 0.0489902893783318, 'weights_max': 0.09459255277172963, 'weights_std': 0.022267354644032733}
[0.3454839686530704] [1260]


### Learn loop

In [87]:
# eval_callback = EvalCallback(eval_env=test_env, best_model_save_path='models/best.stbl',
#                              n_eval_episodes=3, eval_freq=500, render=False, warn=False)
# eval_callback

In [88]:
class MeanRewardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(MeanRewardCallback, self).__init__(verbose)
        self.render = {}

    def _on_step(self) -> bool:

        if self.training_env.get_attr('rewards_memory')[0]:  # else - end of episode
            # reward = self.training_env.get_attr('rewards_memory')[0][-1]
            self.render = self.training_env.render()
        else:
            self.logger.record("train/mean_reward", self.render['mean_reward'])
            self.logger.record("train/min_reward", self.render['min_reward'])
            self.logger.record("train/max_reward", self.render['max_reward'])
            self.logger.record("train/portfolio_value", self.render['portfolio_value'])

        return True

In [89]:
model_name = 'SAC-mr-re-256-256-corrs-dw'
model.learn(total_timesteps=150_000, progress_bar=False, log_interval=1,
            reset_num_timesteps=False, tb_log_name=model_name,
            callback=MeanRewardCallback())

Logging to logs/SAC-mr-re-256-256-corrs-dw_0
{'min_reward': -0.1788921409979694, 'max_reward': 0.08892297520103949, 'mean_reward': 0.0002711987671513675, 'portfolio_value': 0.3417104466107232, 'weights_q0.25': 0.020148079638662754, 'weights_q0.5': 0.031008298210897442, 'weights_q0.75': 0.05254317331350907, 'weights_max': 0.08901289524707764, 'weights_std': 0.01875007540657821}
---------------------------------
| time/              |          |
|    episodes        | 1        |
|    fps             | 148      |
|    time_elapsed    | 8        |
|    total_timesteps | 1260     |
| train/             |          |
|    actor_loss      | -47.3    |
|    critic_loss     | 0.46     |
|    ent_coef        | 0.315    |
|    ent_coef_loss   | -53.6    |
|    learning_rate   | 0.001    |
|    max_reward      | 0.0889   |
|    mean_reward     | 0.000266 |
|    min_reward      | -0.179   |
|    n_updates       | 1159     |
|    portfolio_value | 0.335    |
---------------------------------
{'min_re

{'min_reward': -0.17885128215343749, 'max_reward': 0.1259980923644234, 'mean_reward': 0.00029960163746818743, 'portfolio_value': 0.3774980632099162, 'weights_q0.25': 0.01502203438845991, 'weights_q0.5': 0.03062525712245362, 'weights_q0.75': 0.05189125820344894, 'weights_max': 0.07333904550270333, 'weights_std': 0.022123288943855696}
---------------------------------
| time/              |          |
|    episodes        | 10       |
|    fps             | 133      |
|    time_elapsed    | 94       |
|    total_timesteps | 12600    |
| train/             |          |
|    actor_loss      | -33.3    |
|    critic_loss     | 0.051    |
|    ent_coef        | 0.000514 |
|    ent_coef_loss   | 5.26     |
|    learning_rate   | 0.001    |
|    max_reward      | 0.126    |
|    mean_reward     | 0.00029  |
|    min_reward      | -0.179   |
|    n_updates       | 12499    |
|    portfolio_value | 0.365    |
---------------------------------
{'min_reward': -0.16383123820120393, 'max_reward': 0.

{'min_reward': -0.17069456722223045, 'max_reward': 0.08590975509671253, 'mean_reward': 0.0003268243721808611, 'portfolio_value': 0.4117987089478838, 'weights_q0.25': 0.0252067170377713, 'weights_q0.5': 0.03415465150326273, 'weights_q0.75': 0.0410142446586077, 'weights_max': 0.06916042662412752, 'weights_std': 0.015457435630642398}
---------------------------------
| time/              |          |
|    episodes        | 19       |
|    fps             | 132      |
|    time_elapsed    | 180      |
|    total_timesteps | 23940    |
| train/             |          |
|    actor_loss      | -17.3    |
|    critic_loss     | 0.011    |
|    ent_coef        | 0.000226 |
|    ent_coef_loss   | -12.8    |
|    learning_rate   | 0.001    |
|    max_reward      | 0.0859   |
|    mean_reward     | 0.000321 |
|    min_reward      | -0.171   |
|    n_updates       | 23839    |
|    portfolio_value | 0.404    |
---------------------------------
{'min_reward': -0.18017314337973067, 'max_reward': 0.10

{'min_reward': -0.17865659519297905, 'max_reward': 0.14562616676973522, 'mean_reward': 0.0003682611643490276, 'portfolio_value': 0.46400906707977524, 'weights_q0.25': 0.02045672453129592, 'weights_q0.5': 0.035595033270397064, 'weights_q0.75': 0.04859479027766392, 'weights_max': 0.06774716648036291, 'weights_std': 0.0174308447847073}
---------------------------------
| time/              |          |
|    episodes        | 28       |
|    fps             | 132      |
|    time_elapsed    | 265      |
|    total_timesteps | 35280    |
| train/             |          |
|    actor_loss      | -9.6     |
|    critic_loss     | 0.000623 |
|    ent_coef        | 7.59e-05 |
|    ent_coef_loss   | -63.4    |
|    learning_rate   | 0.001    |
|    max_reward      | 0.146    |
|    mean_reward     | 0.000359 |
|    min_reward      | -0.179   |
|    n_updates       | 35179    |
|    portfolio_value | 0.452    |
---------------------------------
{'min_reward': -0.1797662494113489, 'max_reward': 0.1

{'min_reward': -0.1847251193201035, 'max_reward': 0.1261287602780347, 'mean_reward': 0.0004240401513831691, 'portfolio_value': 0.5342905907427936, 'weights_q0.25': 0.0239575994655211, 'weights_q0.5': 0.03829726706397728, 'weights_q0.75': 0.04739665038164188, 'weights_max': 0.061107547998346584, 'weights_std': 0.015168599417948167}
---------------------------------
| time/              |          |
|    episodes        | 37       |
|    fps             | 132      |
|    time_elapsed    | 351      |
|    total_timesteps | 46620    |
| train/             |          |
|    actor_loss      | -5.2     |
|    critic_loss     | 0.000118 |
|    ent_coef        | 4.19e-05 |
|    ent_coef_loss   | -43.1    |
|    learning_rate   | 0.001    |
|    max_reward      | 0.126    |
|    mean_reward     | 0.000419 |
|    min_reward      | -0.185   |
|    n_updates       | 46519    |
|    portfolio_value | 0.528    |
---------------------------------
{'min_reward': -0.17998635435069274, 'max_reward': 0.14

...

{'min_reward': -0.16329311547622416, 'max_reward': 0.20892302497041898, 'mean_reward': 0.0020550741160210444, 'portfolio_value': 2.5893933861865137, 'weights_q0.25': 0.00940066271644853, 'weights_q0.5': 0.0375506205641063, 'weights_q0.75': 0.060620683507172056, 'weights_max': 0.06456678381647428, 'weights_std': 0.02386728075981224}
---------------------------------
| time/              |          |
|    episodes        | 82       |
|    fps             | 134      |
|    time_elapsed    | 765      |
|    total_timesteps | 103320   |
| train/             |          |
|    actor_loss      | -0.305   |
|    critic_loss     | 8.48e-06 |
|    ent_coef        | 8.04e-06 |
|    ent_coef_loss   | -8.34    |
|    learning_rate   | 0.001    |
|    max_reward      | 0.209    |
|    mean_reward     | 0.00205  |
|    min_reward      | -0.163   |
|    n_updates       | 103219   |
|    portfolio_value | 2.59     |
---------------------------------
{'min_reward': -0.1526045363690866, 'max_reward': 0.20

{'min_reward': -0.148852155155832, 'max_reward': 0.20740296681401243, 'mean_reward': 0.002624007690291726, 'portfolio_value': 3.306249689767573, 'weights_q0.25': 0.010215166089351965, 'weights_q0.5': 0.02402263192654776, 'weights_q0.75': 0.06394093514334284, 'weights_max': 0.06998422799577392, 'weights_std': 0.026101145032296155}
---------------------------------
| time/              |          |
|    episodes        | 91       |
|    fps             | 134      |
|    time_elapsed    | 850      |
|    total_timesteps | 114660   |
| train/             |          |
|    actor_loss      | -0.246   |
|    critic_loss     | 4.81e-05 |
|    ent_coef        | 9.08e-06 |
|    ent_coef_loss   | -13.8    |
|    learning_rate   | 0.001    |
|    max_reward      | 0.207    |
|    mean_reward     | 0.00263  |
|    min_reward      | -0.149   |
|    n_updates       | 114559   |
|    portfolio_value | 3.31     |
---------------------------------
{'min_reward': -0.15640851593448632, 'max_reward': 0.206

{'min_reward': -0.15201122424097915, 'max_reward': 0.20641783033626843, 'mean_reward': 0.0027854052228319685, 'portfolio_value': 3.509610580768279, 'weights_q0.25': 0.009930766441986975, 'weights_q0.5': 0.025923769449914737, 'weights_q0.75': 0.06548775319090627, 'weights_max': 0.07031087003870366, 'weights_std': 0.026267862974110205}
---------------------------------
| time/              |          |
|    episodes        | 100      |
|    fps             | 134      |
|    time_elapsed    | 934      |
|    total_timesteps | 126000   |
| train/             |          |
|    actor_loss      | -0.227   |
|    critic_loss     | 5.56e-06 |
|    ent_coef        | 1.12e-05 |
|    ent_coef_loss   | 21.6     |
|    learning_rate   | 0.001    |
|    max_reward      | 0.206    |
|    mean_reward     | 0.00279  |
|    min_reward      | -0.152   |
|    n_updates       | 125899   |
|    portfolio_value | 3.51     |
---------------------------------
{'min_reward': -0.1520013290900144, 'max_reward': 0.

{'min_reward': -0.15723847207131708, 'max_reward': 0.20682796125848804, 'mean_reward': 0.0030969816821295195, 'portfolio_value': 3.9021969194831927, 'weights_q0.25': 0.013357503530440865, 'weights_q0.5': 0.025289437560607888, 'weights_q0.75': 0.0592321564441895, 'weights_max': 0.07778071011439174, 'weights_std': 0.024697629020929583}
---------------------------------
| time/              |          |
|    episodes        | 109      |
|    fps             | 134      |
|    time_elapsed    | 1018     |
|    total_timesteps | 137340   |
| train/             |          |
|    actor_loss      | -0.247   |
|    critic_loss     | 5.31e-06 |
|    ent_coef        | 1.11e-05 |
|    ent_coef_loss   | -1.17    |
|    learning_rate   | 0.001    |
|    max_reward      | 0.207    |
|    mean_reward     | 0.0031   |
|    min_reward      | -0.157   |
|    n_updates       | 137239   |
|    portfolio_value | 3.9      |
---------------------------------
{'min_reward': -0.1568534224981808, 'max_reward': 0.

{'min_reward': -0.15143566937644126, 'max_reward': 0.20675489053806198, 'mean_reward': 0.00328807773730698, 'portfolio_value': 4.142977949006797, 'weights_q0.25': 0.010930639477993182, 'weights_q0.5': 0.025832406966625525, 'weights_q0.75': 0.0634993178762739, 'weights_max': 0.07523004119948225, 'weights_std': 0.024923456300297974}
---------------------------------
| time/              |          |
|    episodes        | 118      |
|    fps             | 135      |
|    time_elapsed    | 1099     |
|    total_timesteps | 148680   |
| train/             |          |
|    actor_loss      | -0.272   |
|    critic_loss     | 5.43e-06 |
|    ent_coef        | 1.1e-05  |
|    ent_coef_loss   | -15.1    |
|    learning_rate   | 0.001    |
|    max_reward      | 0.207    |
|    mean_reward     | 0.00329  |
|    min_reward      | -0.151   |
|    n_updates       | 148579   |
|    portfolio_value | 4.15     |
---------------------------------
{'min_reward': -0.15388075259987127, 'max_reward': 0.20

#### Saving model and running mean from env

In [90]:
model.save('models/' + model_name)
with open('models/' + model_name + '_obs_rms.pickle', 'wb') as handle:
    pickle.dump(train_env.obs_rms, handle)

## Evaluation

In [91]:
mean_reward, std_reward = evaluate_policy(model, train_env, n_eval_episodes=5, deterministic=False, warn=False, return_episode_rewards=False)
print(mean_reward, std_reward)

{'min_reward': -0.1551756418967425, 'max_reward': 0.2051501027367493, 'mean_reward': 0.0031182704246739334, 'portfolio_value': 3.9290207350891557, 'weights_q0.25': 0.012947083196902343, 'weights_q0.5': 0.02831708697868826, 'weights_q0.75': 0.06261019890797306, 'weights_max': 0.07241900651081343, 'weights_std': 0.023459503237995828}
{'min_reward': -0.15261769454163304, 'max_reward': 0.2067505208626652, 'mean_reward': 0.0031151472645706093, 'portfolio_value': 3.9250855533589637, 'weights_q0.25': 0.011957855735723659, 'weights_q0.5': 0.023034958578173094, 'weights_q0.75': 0.06179814493697238, 'weights_max': 0.06960267782515266, 'weights_std': 0.024291828874461626}
{'min_reward': -0.1558077356194606, 'max_reward': 0.2057931701088344, 'mean_reward': 0.003103618810809518, 'portfolio_value': 3.9105597016199978, 'weights_q0.25': 0.013518319494265381, 'weights_q0.5': 0.02403929192007034, 'weights_q0.75': 0.0625736048484683, 'weights_max': 0.07221934615278286, 'weights_std': 0.023597901522330263

In [92]:
train_env.venv.venv.envs[0].__dict__.keys()

dict_keys(['data', 'stock_dim', 'close_prices_pct', 'tickers', 'n_assets', 'objective', 'risk_measure', 'rewards_memory', 'actions_memory', 'portfolio_value', 'observations_range', 'current_step', 'weights', 'reward_range', 'action_space', 'observation_space'])

In [93]:
train_env.training = False
train_env.venv.venv.envs[0].observations_range = (0, 1760) # change to use the test period

In [94]:
import copy

obs = train_env.reset()
weights_states = [train_env.venv.venv.envs[0].weights]
portfolio_values = [0]
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = train_env.step(action)
    if done:
        break
    else:
        weights_states.append(copy.deepcopy(train_env.venv.venv.envs[0].weights))
        portfolio_values.append(train_env.venv.venv.envs[0].portfolio_value)
        print(train_env.render())
len(portfolio_values), len(weights_states)

{'min_reward': -0.004746610762858191, 'max_reward': -0.004746610762858191, 'mean_reward': -0.004746610762858191, 'portfolio_value': -0.004746610762858191, 'weights_q0.25': 0.012425719301477869, 'weights_q0.5': 0.028633291510034624, 'weights_q0.75': 0.06033297567917744, 'weights_max': 0.0743449346773447, 'weights_std': 0.02318512511726389}
{'min_reward': -0.021275502604930344, 'max_reward': -0.004746610762858191, 'mean_reward': -0.013011056683894268, 'portfolio_value': -0.026022113367788535, 'weights_q0.25': 0.013377119957668228, 'weights_q0.5': 0.031773963374719216, 'weights_q0.75': 0.05709768210254099, 'weights_max': 0.06952922760609104, 'weights_std': 0.022144421160463296}
{'min_reward': -0.021275502604930344, 'max_reward': -0.004746610762858191, 'mean_reward': -0.011356544208212463, 'portfolio_value': -0.034069632624637386, 'weights_q0.25': 0.012924303139651544, 'weights_q0.5': 0.029190664871458286, 'weights_q0.75': 0.06026878612609565, 'weights_max': 0.07495704238045953, 'weights_s

{'min_reward': -0.02742439463692864, 'max_reward': 0.03021282574113266, 'mean_reward': 0.002840218400381273, 'portfolio_value': 0.19881528802668913, 'weights_q0.25': 0.009735467922494575, 'weights_q0.5': 0.0339060874691871, 'weights_q0.75': 0.061292301205141134, 'weights_max': 0.06608316991249154, 'weights_std': 0.02445705229029152}
{'min_reward': -0.02742439463692864, 'max_reward': 0.03021282574113266, 'mean_reward': 0.002898392869942885, 'portfolio_value': 0.20578589376594483, 'weights_q0.25': 0.013803453327223665, 'weights_q0.5': 0.01516560655750862, 'weights_q0.75': 0.05842922039813853, 'weights_max': 0.09337310120878424, 'weights_std': 0.02832481801502476}
{'min_reward': -0.02742439463692864, 'max_reward': 0.03021282574113266, 'mean_reward': 0.0028748182885622205, 'portfolio_value': 0.20698691677647987, 'weights_q0.25': 0.010330511921364458, 'weights_q0.5': 0.02502540362918424, 'weights_q0.75': 0.06645911211235275, 'weights_max': 0.06917215646639038, 'weights_std': 0.0253495441760

{'min_reward': -0.02742439463692864, 'max_reward': 0.03021282574113266, 'mean_reward': 0.003391934218139799, 'portfolio_value': 0.4341675799218941, 'weights_q0.25': 0.015609044222964418, 'weights_q0.5': 0.016667422883993543, 'weights_q0.75': 0.02486776577886215, 'weights_max': 0.12194339313739927, 'weights_std': 0.036939187710260175}
{'min_reward': -0.02742439463692864, 'max_reward': 0.03021282574113266, 'mean_reward': 0.0034345555242686083, 'portfolio_value': 0.4430576626306503, 'weights_q0.25': 0.014841441995471813, 'weights_q0.5': 0.017684388525860732, 'weights_q0.75': 0.039980016793115795, 'weights_max': 0.11053507943879695, 'weights_std': 0.03229491671610776}
{'min_reward': -0.02742439463692864, 'max_reward': 0.03021282574113266, 'mean_reward': 0.003471291908605007, 'portfolio_value': 0.4512679481186507, 'weights_q0.25': 0.015199403846139409, 'weights_q0.5': 0.016137013948930995, 'weights_q0.75': 0.03217080446132973, 'weights_max': 0.11884460883855893, 'weights_std': 0.03795392024

{'min_reward': -0.03779753005124128, 'max_reward': 0.03021282574113266, 'mean_reward': 0.0024547132811090803, 'portfolio_value': 0.486033229659598, 'weights_q0.25': 0.013634260142504597, 'weights_q0.5': 0.021619545412943805, 'weights_q0.75': 0.05338338072262827, 'weights_max': 0.0867497003342449, 'weights_std': 0.026236052685693134}
{'min_reward': -0.03779753005124128, 'max_reward': 0.03021282574113266, 'mean_reward': 0.0024710344155239655, 'portfolio_value': 0.4917358486892693, 'weights_q0.25': 0.0138261119851038, 'weights_q0.5': 0.022761098607861695, 'weights_q0.75': 0.04950371996540579, 'weights_max': 0.09242541508657408, 'weights_std': 0.027516228773693096}
{'min_reward': -0.03779753005124128, 'max_reward': 0.03021282574113266, 'mean_reward': 0.002491734908549388, 'portfolio_value': 0.49834698170987773, 'weights_q0.25': 0.013942028599395544, 'weights_q0.5': 0.019022172910399548, 'weights_q0.75': 0.05291444440805548, 'weights_max': 0.09362578012732083, 'weights_std': 0.0287930311838

{'min_reward': -0.03779753005124128, 'max_reward': 0.03021282574113266, 'mean_reward': 0.0024950513413680496, 'portfolio_value': 0.6387331433902206, 'weights_q0.25': 0.012111720273221991, 'weights_q0.5': 0.013965906800003168, 'weights_q0.75': 0.06726447363376134, 'weights_max': 0.09124373902796619, 'weights_std': 0.030910107292910417}
{'min_reward': -0.03779753005124128, 'max_reward': 0.03021282574113266, 'mean_reward': 0.0024925047078697463, 'portfolio_value': 0.6405737099225247, 'weights_q0.25': 0.012759861743216562, 'weights_q0.5': 0.01607133521256751, 'weights_q0.75': 0.07181560905750992, 'weights_max': 0.09081903801737456, 'weights_std': 0.030528636405699066}
{'min_reward': -0.03779753005124128, 'max_reward': 0.03021282574113266, 'mean_reward': 0.002507970992158203, 'portfolio_value': 0.6470565159768162, 'weights_q0.25': 0.012119863927937005, 'weights_q0.5': 0.019673877552485106, 'weights_q0.75': 0.06171339819513307, 'weights_max': 0.08186515542491504, 'weights_std': 0.02684061052

...

{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.0027806594564589587, 'portfolio_value': 4.2182603954482465, 'weights_q0.25': 0.011950886707238772, 'weights_q0.5': 0.020765475415983926, 'weights_q0.75': 0.06227287491181196, 'weights_max': 0.08101298461671758, 'weights_std': 0.0270075646347273}
{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.002773486862860712, 'portfolio_value': 4.210153057822568, 'weights_q0.25': 0.012093554155611651, 'weights_q0.5': 0.013085875950152046, 'weights_q0.75': 0.07713977016267157, 'weights_max': 0.08933297111833483, 'weights_std': 0.031720273376345226}
{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.002764074043456422, 'portfolio_value': 4.198628472010312, 'weights_q0.25': 0.011267627468512637, 'weights_q0.5': 0.013800989921474164, 'weights_q0.75': 0.07752501604318099, 'weights_max': 0.0865200757806375, 'weights_std': 0.0318201858672156

{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.002684850318064945, 'portfolio_value': 4.255487754132945, 'weights_q0.25': 0.01228064550979591, 'weights_q0.5': 0.020692325560682626, 'weights_q0.75': 0.06406784088174543, 'weights_max': 0.08650472925291784, 'weights_std': 0.02818462349827635}
{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.0026725439320830174, 'portfolio_value': 4.238654676283673, 'weights_q0.25': 0.011241042277233266, 'weights_q0.5': 0.016118811206600324, 'weights_q0.75': 0.0699017838016866, 'weights_max': 0.0820755511541388, 'weights_std': 0.029004827747906022}
{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.0026677066870437823, 'portfolio_value': 4.23365051233849, 'weights_q0.25': 0.013680571134702387, 'weights_q0.5': 0.02845238815128802, 'weights_q0.75': 0.05764775143338621, 'weights_max': 0.07812505007605654, 'weights_std': 0.02362374499567054}


{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.0024663207209540414, 'portfolio_value': 4.044765982364633, 'weights_q0.25': 0.015071252466796806, 'weights_q0.5': 0.02558231402515066, 'weights_q0.75': 0.05598669146351233, 'weights_max': 0.08781130137599338, 'weights_std': 0.022760675558924748}
{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.0024608513139771893, 'portfolio_value': 4.038257006236573, 'weights_q0.25': 0.01674211766672966, 'weights_q0.5': 0.03220314281807489, 'weights_q0.75': 0.04921880959908648, 'weights_max': 0.07548683748389859, 'weights_std': 0.020749989026735922}
{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.0024510387285807357, 'portfolio_value': 4.024605592329574, 'weights_q0.25': 0.013319689828614787, 'weights_q0.5': 0.028028167610569733, 'weights_q0.75': 0.053254339686394904, 'weights_max': 0.07740649479489846, 'weights_std': 0.02267531378162

{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.0022917026599796513, 'portfolio_value': 3.916519845905231, 'weights_q0.25': 0.013718326677922301, 'weights_q0.5': 0.020437821368521528, 'weights_q0.75': 0.06583406287581853, 'weights_max': 0.09296189809327525, 'weights_std': 0.028248656716715663}
{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.002275769921243149, 'portfolio_value': 3.891566565325792, 'weights_q0.25': 0.013638092217791647, 'weights_q0.5': 0.02178937026407047, 'weights_q0.75': 0.04781804207890979, 'weights_max': 0.09381659107097808, 'weights_std': 0.026758365302367706}
{'min_reward': -0.15280873134165546, 'max_reward': 0.20630300645779137, 'mean_reward': 0.0022712914580730634, 'portfolio_value': 3.8861796847630186, 'weights_q0.25': 0.009911056061248511, 'weights_q0.5': 0.028048781344622638, 'weights_q0.75': 0.06339240827914856, 'weights_max': 0.07269055317891662, 'weights_std': 0.0253483716754

(1760, 1760)

In [95]:
portfolio_state = pd.DataFrame(index=train_env.venv.venv.envs[0].close_prices_pct.index, data=portfolio_values)
portfolio_state.to_csv('portfolios/' + model_name + '_portfolio.csv')

In [96]:
weights_df = pd.DataFrame(index=train_env.venv.venv.envs[0].close_prices_pct.columns, data=weights_states[-1])
weights_df.to_csv('portfolios/' + model_name + '_weights.csv')

## Tests

In [294]:
with open('data/sp500_data_5y.pickle', 'rb') as f:
    data = pickle.load(f)
    close_prices = pd.DataFrame({k: v['Close'] for k, v in data.items()})

FileNotFoundError: [Errno 2] No such file or directory: 'data/sp500_data_5y.pickle'

In [ ]:
data['AAPL'].diff()

In [ ]:
rand_w = softmax_normalization(np.random.randn(489))
rand_w

In [ ]:
ones_w = softmax_normalization(np.ones(489))
ones_w

In [ ]:
close_prices.diff()

In [ ]:
(close_prices.pct_change().iloc[48] @ rand_w)

In [ ]:
close_prices.pct_change().dropna().mean() @ rand_w

In [ ]:
(close_prices.pct_change().dropna() @ rand_w).sum()

In [ ]:
(close_prices.pct_change().dropna() @ ones_w).sum()